In [1]:
TWITTER_BEARER_TOKEN='AAAAAAAAAAAAAAAAAAAAAAbGOQEAAAAAmab6fax484aouKaxR5b71oliE4M%3DxHyswRUwHfnKGFR0XOvPXZQulqdYPZByI6Js2cPhB85Tln4nAB'

In [66]:
import tweepy

client = tweepy.Client(TWITTER_BEARER_TOKEN, wait_on_rate_limit=True)


In [267]:
def get_likes_for_tweet(tweet_id):
    liking_users =  client.get_liking_users(tweet_id).data
    if not liking_users:
        return []
    return liking_users

In [57]:
def get_retweets_for_tweet(tweet_id):
    retweeting_users = client.get_retweeters(tweet_id).data
    return retweeting_users


In [1192]:
def get_commenters_for_tweet(tweet_id):
    commenting_users = client.search_recent_tweets(query=f"conversation_id:{tweet_id}", max_results=100, tweet_fields="public_metrics,entities,created_at,author_id").data
    if not commenting_users:
        return []
    filtered_users = [user for user in commenting_users if user is not None]
    return filtered_users

In [114]:
def get_quotes_of_tweet(tweet_id):
    quoting_users = client.search_recent_tweets(query=f"url:{tweet_id} is:quote", max_results=100, tweet_fields="public_metrics,entities,created_at,author_id").data
    if not quoting_users:
        return []
    filtered_users = [user.author_id for user in quoting_users if user is not None]
    return filtered_users

In [118]:
def print_quotes_for_user(username):
    quoting_users = client.search_recent_tweets(query=f"url:{username} is:quote", max_results=100, tweet_fields="public_metrics,entities,created_at,author_id").data
    print(quoting_users)
    if not quoting_users:
        return []
    filtered_users = [user.author_id for user in quoting_users if user is not None]
    return filtered_users

In [1302]:
USER = "MeetKevon"

In [1304]:
initial_data = client.search_recent_tweets(query=f'from:{USER}', max_results=100, tweet_fields="public_metrics,entities,created_at,author_id")
recent_tweets = initial_data.data
next_token = initial_data.meta.get("next_token", "")

In [1305]:
while(next_token):
    print(next_token)
    print(len(recent_tweets))
    data = client.search_recent_tweets(query=f'from:{USER}', max_results=100, tweet_fields="public_metrics,entities,created_at,author_id", next_token=next_token)
    print(len(data.data))
    recent_tweets.extend(data.data)
    next_token=data.meta.get("next_token", "")

b26v89c19zqg8o3fpe7705aa8qibkrcd6cpp327syz2bh
100
100
b26v89c19zqg8o3fpe76kzond9btlxu04man6kytkjkvx
200
100
b26v89c19zqg8o3fpe76kimu0hftnl5h8r6n8dypan6v1
300
16


In [1306]:
likes = []
retweets = []
comments = []
quotes = []


for tweet in recent_tweets:
    if tweet.public_metrics["retweet_count"] > 0:
        retweets.append(tweet)
    if tweet.public_metrics["reply_count"] > 0:
        comments.append(tweet)
    if tweet.public_metrics["like_count"] > 0:
        likes.append(tweet)
    if tweet.public_metrics["quote_count"] > 0:
        quotes.append(tweet)


In [1307]:
liking_users = []
processed = []


In [1310]:
for like in likes:
    found = [item for item in processed if item.id == like.id]
    if not found:
        liking_users.extend(get_likes_for_tweet(like.id))
        processed.append(like)
    

Rate limit exceeded. Sleeping for 890 seconds.


In [1039]:
print(len(likes))
print(len(processed))
print(len(recent_tweets))

559
384
756


In [1311]:
retweeting_users = []
processed_retweets = []


In [1313]:
for retweet in retweets:
    found = [item for item in processed_retweets if item.id == retweet.id]
    if not found:
        rts = get_retweets_for_tweet(retweet.id)
        if rts:
            retweeting_users.extend(rts)
        processed_retweets.append(retweet)

In [754]:
print(len(retweets))
print(len(processed_retweets))

115
75


In [1314]:
commenting_users = []
processed_comments = []



In [1315]:
for comment in comments:
    found = [item for item in processed_comments if item.id == comment.id]
    if not found:
        commenting_users.extend(get_commenters_for_tweet(comment.id))
        processed_comments.append(comment)

In [621]:
quoting_users = []
for quote in quotes:
    print(get_quotes_of_tweet(quote.id))

In [1316]:
stats = {}

In [1317]:
for user in liking_users:
    if not stats.get(user.id):
        stats[user.id] = {
            "name": user.name,
            "username": user.username
        }
    
    stats[user.id]["likes"] = [liking_user.id for liking_user in liking_users].count(user.id)

In [1318]:
for user in retweeting_users:
    if not stats.get(user.id):
        stats[user.id] = {
            "name": user.name,
            "username": user.username
        }
    
    stats[user.id]["retweets"] = [retweeting_user.id for retweeting_user in retweeting_users].count(user.id)

In [1319]:
for user in commenting_users:
    if not stats.get(user):
        stats[user] = {}
    
    stats[user]["comments"] = commenting_users.count(user)

In [1320]:

{k: v for k, v in sorted(stats.items(), key=lambda item: (item[1].get("likes", 0) + (2 * item[1].get("retweets", 0)) + 3 * item[1].get("comments", 0)), reverse= True)}
                         
                         

{1121648833002262528: {'name': '🦓 Leif Latiff',
  'username': 'leifjerami',
  'likes': 34},
 2904219673: {'name': 'Mike Cardona 🤖',
  'username': 'CSMikeCardona',
  'likes': 20},
 2230269272: {'name': 'Dr. Melvin Varghese 🥗',
  'username': 'mvarghese5',
  'likes': 20},
 896718379075096578: {'name': 'SAM | SMM/VA',
  'username': 'nonpartiSAM',
  'likes': 12,
  'retweets': 4},
 132235427: {'name': 'Mattia Righetti 🐙',
  'username': 'heymattia',
  'likes': 18,
  'retweets': 1},
 28257675: {'name': 'Marie Ng 🦙🌳', 'username': 'threehourcoffee', 'likes': 17},
 1426510246357385216: {'name': 'Sanja Williams',
  'username': 'SanjaWilliams1',
  'likes': 15},
 1470485913507663877: {'name': 'Christina Konstantinidou ✨🚢 ✏️🏴\u200d☠️',
  'username': 'ChrKonst',
  'likes': 15},
 1463434417490194436: {'name': 'Sapna Devi',
  'username': 'Sapdevi',
  'likes': 14},
 777576246171541505: {'name': 'Onur Genes 🐺',
  'username': 'onurgenes',
  'likes': 13},
 1126179933523464198: {'name': 'Seth Fannin',
  'use

In [1321]:
engaged_users = len(stats)
engaged_users

999

In [1322]:
scores = []
for key,value in stats.items():
    scores.append(value.get("likes", 0) + (2 * value.get("retweets", 0)) + 3 * value.get("comments", 0))


In [1323]:
length = len(scores)
total = sum(scores)
avg = total/length

In [1324]:
avg

2.7477477477477477

In [1325]:
sum(sorted(scores, reverse=True)[:10])/10 if length > 10 else avg

18.8

In [633]:
audience_width = [4.3, 2.8, 7.1, 5.2, 4.7, 1.2, 2.8, 11.0]
print(sum(audience_width)/len(audience_width))

4.014285714285714


In [ ]:
Ryan:
57/1321=4.3%

55/1321

3.807017543859649 avg score
3.7818181818181817
11.9 - how engaged are your top 10 fans
11.7


Manish:
12/436=2.8%
1.3333333333333333
1.4


Oliriev:
37/521 = 7.1%
2.108108108108108
4.4

43/521 = 17.1%
3.7674418604651163
11.9

David:
56/1088 = 5.1%
3.4464285714285716
12.7

57/1088 = 5.2%
3.456140350877193
12.8

Sydney:
12/16
3.0
3.4


Ben:
94/2517 = 3.7%
1.6595744680851063
5.6

119/2517 = 4.7%
1.7226890756302522
7.1


Tommy:
92/7598 = 1.2%
3.2065217391304346
14.6

112/7598 = 1.5%
3.4375
19.6


Jon:
147/367 = 40%
1.0816326530612246
2.2

160/367 = 43.6%
1.10625
2.3



Justin:
83/2926 = 2.8%
1.5060240963855422
3.3

69/2926
1.565217391304348
3.7

Rashik
15/229 = 
1.2666666666666666
1.4

Dagobert
2073/18800 = 11%
3.4370477568740956
164.2

Mike
290/1918 = 15.1%
4.686206896551724
59.5

Alex
186/1101 = 16.9%
2.7096774193548385
22.8

JC
236/1333 = 17.7%
2.48728813559322
22.2

Bhushan
43/118 = 36.4%
4.465116279069767
15.1

Brian
119/2156 = 5.5%
3.310924369747899
13.8

Shawna
74/7459 = 1.0%
2.445945945945946
8.9

Saps
24/1593 = 1.5%
2.2083333333333335
3.9

Olu
698/3205 = 21.8%
2.836676217765043
37.9

Sven
242/556 = 43.5%
4.148760330578512
41.4

Lit_Check
4/100 = 4.0%
2.5
2.5

Ed
280/ 2677 = 10.4%
3.9035714285714285
39.8

Jenn
431/3806 = 11.3%
5.821345707656612
102.8

Suhail
1232/15000 = 8.2%
2.8125
59.6

Jay
30/108 = 27.8%
4.0
10.0

Lenin
513/892 = 57.5%
2.7660818713450293
37.7

Sajal
2145/5230 = 41.0%
3.694172494172494
172.1

Walid
120/106 = 113.2%
2.283333333333333
12.5

Jibran
100/279 = 35.8%
3.55
20.9

Ana
548/2676 = 20.4%
5.083941605839416
98.6


Leo
103/
5.601941747572815
32.3


140/
1.4214285714285715
4.8

Jess
31/
1.3225806451612903
2.0


Niharika
1398/4829 = 29.0%
2.8834048640915593
30.3

ADHDJobs (Kim)
579/1090 = 53.1%
2.8583765112262522
20.0

Jackie
44/22900 = 0.2%
1.1136363636363635
1.5


1004
1.7709163346613546
4.4

Kevon
999/9121
2.7477477477477477
18.8

In [634]:
engagement_averages = [1.2666666666666666, 1.565217391304348, 1.5060240963855422, 1.10625, 3.4375, 1.7226890756302522, 3.0, 3.456140350877193, 3.7674418604651163, 1.3333333333333333, 3.7818181818181817,3.4370477568740956
]
print(sum(engagement_averages)/len(engagement_averages))

2.358461905134603


In [635]:
superfan_index = [1.4, 3.7, 3.3, 2.3, 2.2,19.6, 7.1, 3.4, 12.8, 11.9, 164.2]
print(sum(superfan_index)/len(superfan_index))

6.7700000000000005


In [1171]:
analyzed_accounts = [
    {
        "name": "Ryan",
        "audience_fit": 4.3,
        "average_engagement": 3.7818181818181817,
        "superfan_index": 11.7
        "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Manish",
        "audience_fit": 2.8,
        "average_engagement": 1.3333333333333333,
        "superfan_index": 1.4,
                "tags": [],
        "followers": 1,
        "following": 1
    },

    {
        "name": "Oliriev",
        "audience_fit": 17.1,
        "average_engagement": 3.7674418604651163,
        "superfan_index": 11.9,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "David",
        "audience_fit": 5.2,
        "average_engagement": 3.456140350877193,
        "superfan_index": 12.8,
                "tags": [],
        "followers": 1,
        "following": 1
    },

    {
        "name": "Sydney",
        "audience_fit": 75.0,
        "average_engagement": 3.0,
        "superfan_index": 3.4,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Ben",
        "audience_fit": 4.7,
        "average_engagement": 1.7226890756302522,
        "superfan_index": 7.1,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Tommy",
        "audience_fit": 1.5,
        "average_engagement": 3.4375,
        "superfan_index": 19.6,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Jon",
        "audience_fit": 43.6,
        "average_engagement": 1.10625,
        "superfan_index": 2.3,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Justin",
        "audience_fit": 2.8,
        "average_engagement": 1.565217391304348,
        "superfan_index": 3.7,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Rashik",
        "audience_fit": 6.6,
        "average_engagement": 1.2666666666666666,
        "superfan_index": 1.4,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Dagobert",
        "audience_fit": 11.0,
        "average_engagement": 3.4370477568740956,
        "superfan_index": 164.2,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Mike",
        "audience_fit": 15.1,
        "average_engagement": 4.686206896551724,
        "superfan_index": 59.5,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Alex",
        "audience_fit": 16.9,
        "average_engagement": 2.7096774193548385,
        "superfan_index": 22.8,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "JC",
        "audience_fit": 17.7,
        "average_engagement": 2.48728813559322,
        "superfan_index": 22.2,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Bhushan",
        "audience_fit": 36.4,
        "average_engagement": 4.465116279069767,
        "superfan_index": 15.1,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Brian",
        "audience_fit": 5.5,
        "average_engagement": 3.310924369747899,
        "superfan_index": 13.8,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Shawna",
        "audience_fit": 1.0,
        "average_engagement": 2.445945945945946,
        "superfan_index": 8.9,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Saps",
        "audience_fit": 1.5,
        "average_engagement": 2.2083333333333335,
        "superfan_index": 3.9,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Olu",
        "audience_fit": 21.8,
        "average_engagement": 2.836676217765043,
        "superfan_index": 37.9,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Sven",
        "audience_fit": 43.5,
        "average_engagement": 4.148760330578512,
        "superfan_index": 41.4,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Lit_Check",
        "audience_fit": 4.0,
        "average_engagement": 2.5,
        "superfan_index": 2.5,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Ed",
        "audience_fit": 10.4,
        "average_engagement": 3.9035714285714285,
        "superfan_index": 39.8,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Jenn",
        "audience_fit": 11.3,
        "average_engagement": 5.821345707656612,
        "superfan_index": 102.8,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Suhail",
        "audience_fit": 8.2,
        "average_engagement": 2.8125,
        "superfan_index": 59.6,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Jay",
        "audience_fit": 27.8,
        "average_engagement": 4.0,
        "superfan_index": 10.0,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Lenin",
        "audience_fit": 57.5,
        "average_engagement": 2.7660818713450293,
        "superfan_index": 37.7,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Sajal",
        "audience_fit": 41.0,
        "average_engagement": 3.694172494172494,
        "superfan_index": 172.1,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Walid",
        "audience_fit": 113.2,
        "average_engagement": 2.283333333333333,
        "superfan_index": 12.5,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Jibran",
        "audience_fit": 35.8,
        "average_engagement": 3.55,
        "superfan_index": 20.9,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Ana",
        "audience_fit": 20.4,
        "average_engagement": 5.1,
        "superfan_index": 98.6,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Leo",
        "audience_fit": 6.8,
        "average_engagement": 5.601941747572815,
        "superfan_index": 32.3,
                "tags": [],
        "followers": 1,
        "following": 1
    },
    {
        "name": "Arthur",
        "audience_fit": 7.0,
        "average_engagement": 1.4214285714285715,
        "superfan_index": 4.8,
                "tags": [],
        "followers": 1,
        "following": 1
    },
]



















In [1175]:
import statistics


In [1187]:
sfa_median = statistics.median([account["superfan_index"] for account in analyzed_accounts])
sfa_mean = statistics.mean([account["superfan_index"] for account in analyzed_accounts])
sfa_variance = statistics.variance([account["superfan_index"] for account in analyzed_accounts])
sfa_stddev = statistics.stdev([account["superfan_index"] for account in analyzed_accounts])
sfa_min = min([account["superfan_index"] for account in analyzed_accounts])
sfa_max = max([account["superfan_index"] for account in analyzed_accounts])

ae_median = statistics.median([account["average_engagement"] for account in analyzed_accounts])
ae_mean = statistics.mean([account["average_engagement"] for account in analyzed_accounts])
ae_variance = statistics.variance([account["average_engagement"] for account in analyzed_accounts])
ae_stddev = statistics.stdev([account["average_engagement"] for account in analyzed_accounts])
ae_min = min([account["average_engagement"] for account in analyzed_accounts])
ae_max = max([account["average_engagement"] for account in analyzed_accounts])

af_median = statistics.median([account["audience_fit"] for account in analyzed_accounts])
af_mean = statistics.mean([account["audience_fit"] for account in analyzed_accounts])
af_variance = statistics.variance([account["audience_fit"] for account in analyzed_accounts])
af_stddev = statistics.stdev([account["audience_fit"] for account in analyzed_accounts])
af_min = min([account["audience_fit"] for account in analyzed_accounts])
af_max = max([account["audience_fit"] for account in analyzed_accounts])

In [1188]:
print(sfa_median)
print(sfa_mean)
print(sfa_variance)
print(sfa_stddev)
print(sfa_min)
print(sfa_max)

14.45
33.08125
1912.9260887096773
43.73701051409066
1.4
172.1


In [1189]:
print(ae_median)
print(ae_mean)
print(ae_variance)
print(ae_stddev)
print(ae_min)
print(ae_max)

3.1554621848739495
3.14460652184343
1.515489769843079
1.2310523018308681
1.10625
5.821345707656612


In [1190]:
print(af_median)
print(af_mean)
print(af_variance)
print(af_stddev)
print(af_min)
print(af_max)

11.15
21.16875
611.8738306451613
24.736083575318897
1.0
113.2
